In [2]:
import base64
import argparse
from pathlib import Path

In [3]:

TEMPLATE = """<!DOCTYPE html>
<html>
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="icon" href="https://www.dashboardtify.com/favicon.svg" type="image/x-icon">
    <title>{title} dashboardtify</title>
    <link href="https://www.dashboardtify.com/gridstack/gridstack.min.css" rel="stylesheet">
    <script src="https://www.dashboardtify.com/gridstack/gridstack-all.js"></script>
    <script src="https://code.iconify.design/1/1.0.6/iconify.min.js"></script>
    <link href="https://www.dashboardtify.com/1.6/dashboardtify.css" rel="stylesheet">
    <script src="https://www.dashboardtify.com/1.6/dashboardtify.js" defer></script>
  </head>
  <body>
    <div id="loader-container" class="loader-container">
      <div id="loader" class="loader"></div>
    </div>

    <script>
      // Các biến config gốc
      var fileName = '{file_name}';
      var savedData = [];
      var edit = false;
    </script>

    <script>
      // Base64 của notebook HTML được nhúng trực tiếp
      var notebookBase64 = "{notebook_base64}";

      // Biến urlNotebook dùng data URL thay vì phải upload lên mạng
      var urlNotebook = "data:text/html;base64," + notebookBase64;
    </script>
  </body>
</html>
"""

In [4]:
def build_html(notebook_path: Path, output_path: Path, title: str | None = None):
    if not notebook_path.is_file():
        raise FileNotFoundError(f"Không tìm thấy file notebook: {notebook_path}")

    # Đọc file notebook dạng bytes rồi encode Base64
    notebook_bytes = notebook_path.read_bytes()
    notebook_b64 = base64.b64encode(notebook_bytes).decode("utf-8")

    # Tên file (không phần mở rộng) dùng cho fileName và title mặc định
    stem = notebook_path.stem
    if title is None:
        title = stem

    html_content = TEMPLATE.format(
        title=title,
        file_name=stem,
        notebook_base64=notebook_b64,
    )

    # Ghi ra file HTML output
    output_path.write_text(html_content, encoding="utf-8")
    print(f"Đã tạo file HTML tự chứa tại: {output_path}")


def main():
    parser = argparse.ArgumentParser(
        description="Nhúng notebook HTML local (Dashboardtify) thành 1 file HTML duy nhất dùng Base64."
    )
    parser.add_argument(
        "notebook_html",
        help="Đường dẫn tới file notebook HTML local (vd: iowa_liquor_retail_sales.html)",
    )
    parser.add_argument(
        "-o",
        "--output",
        help="Đường dẫn file HTML output (mặc định: <stem>_embedded.html)",
        default=None,
    )
    parser.add_argument(
        "-t",
        "--title",
        help="Title hiển thị trên tab trình duyệt (mặc định: lấy theo tên file notebook)",
        default=None,
    )

    args = parser.parse_args()

    notebook_path = Path(args.notebook_html).expanduser().resolve()

    if args.output is None:
        output_path = notebook_path.with_name(f"{notebook_path.stem}_embedded.html")
    else:
        output_path = Path(args.output).expanduser().resolve()

    build_html(notebook_path, output_path, title=args.title)

In [5]:
notebook_path = Path("pipeline-02-binarize_data_ver_3.html").expanduser().resolve()


output_path = notebook_path.with_name(f"pipeline-02-binarize_data_ver_3_embedded.html")


build_html(notebook_path, output_path, title="pipeline-02-binarize_data_ver_3")

Đã tạo file HTML tự chứa tại: D:\Workspace\dashboard\pipeline-02-binarize_data_ver_3_embedded.html
